<a href="https://colab.research.google.com/github/Jungsan005/Bigdata-Class/blob/master/%EC%95%84%ED%8C%8C%ED%8A%B8%EB%A7%A4%EB%A7%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1-1. 데이터 가져오기
import requests
import datetime

url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?"
service_key = "G4mFbypnd1vbyqajV%2B8CNMlgLri6mbVRPrTld3%2Bfxj7xhcRUlwYBbDCWZpJ9MZcJYbjdWK%2FZYgKen57sHxpw7A%3D%3D"
base_date = "201001" 
gu_code = '11000' ## 법정동 코드 5자리라면, 구 단위로 데이터를 확보하는 것. 11215 = 광진구

payload = "LAWD_CD=" + gu_code + "&" + \
          "DEAL_YMD=" + base_date + "&" + \
          "serviceKey=" + service_key + "&" 
          
res = requests.get(url + payload)
print(res)

<Response [200]>


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def get_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list
    
items_list = get_items(res)
items = pd.DataFrame(items_list) 
items.head()

""


In [ ]:
code_file = "https://raw.githubusercontent.com/Jungsan005/Bigdata-Class/master/datasets/%EB%B2%95%EC%A0%95%EB%8F%99%EC%BD%94%EB%93%9C%20%EC%A0%84%EC%B2%B4%EC%9E%90%EB%A3%8C.txt"
code = pd.read_csv(code_file, sep='\t')
code.columns = ['code', 'name', 'is_exist']
code

,code,name,is_exist
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46175,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46176,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46177,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46178,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [ ]:
code = code [code['is_exist'] == '존재']
code

,code,name,is_exist
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46175,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46176,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46177,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46178,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [ ]:
print(code['code'][0])
print(type(code['code'][0])) ## int64타입

## string으로 변경
code['code'] = code['code'].apply(str) 

1100000000
<class 'numpy.int64'>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
year = [str("%02d" %(y)) for y in range(2010, 2020)]
month = [str("%02d" %(m)) for m in range(1, 13)]
base_date_list = ["%s%s" %(y, m) for y in year for m in month ]

In [49]:
gu = "강북구"
gu_code = code[ (code['name'].str.contains(gu) )]
gu_code = gu_code['code'].reset_index(drop=True)
gu_code = str(gu_code[0])[0:5]
print(gu_code)

11305


In [50]:
import requests
import datetime

def get_data(gu_code, base_date):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?"
    service_key = "G4mFbypnd1vbyqajV%2B8CNMlgLri6mbVRPrTld3%2Bfxj7xhcRUlwYBbDCWZpJ9MZcJYbjdWK%2FZYgKen57sHxpw7A%3D%3D"    
    payload = "LAWD_CD=" + gu_code + "&" + \
              "DEAL_YMD=" + base_date + "&" + \
              "serviceKey=" + service_key + "&"

    res = requests.get(url + payload)
    
    return res

In [52]:
items_list = []
for base_date in base_date_list:
    res = get_data(gu_code, base_date)
    items_list += get_items(res)
    
len(items_list)

AttributeError: ignored

In [48]:
import os

items = pd.DataFrame(items_list) 
items.head()
items.to_csv(os.path.join("%s_%s~%s.csv" %(gu, year[0], year[-1])), index=False,encoding="euc-kr") 